# Cleaning Datasets

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import datetime as dt

Reading in data frames

In [2]:
df_china = pd.read_csv('ETFs/CHINA - MCHI.csv')
df_india = pd.read_csv('ETFs/INDIA-Motilal Oswal NASDAQ 100 ETF.csv')
df_SA = pd.read_csv('ETFs/SA - SRSA.L.csv')
df_russia = pd.read_csv('ETFs/RUSSIA - iShares MSCI Russia ADDR GDR UCITS ETF USD.csv') 
df_brazil = pd.read_csv('ETFs/BRAZIL - iShares MSCI Brazil ETF (EWZ).csv')
df_hist = pd.read_csv('HistoricalPrices.csv')

In [3]:
df_china.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2011-03-31,51.689999,51.950001,51.590000,51.900002,42.296864,14200
1,2011-04-01,52.580002,52.720001,52.529999,52.619999,42.883629,2900
2,2011-04-04,53.779999,53.779999,53.119999,53.740002,43.796398,10500
3,2011-04-05,53.500000,53.560001,53.400002,53.419998,43.535599,3700
4,2011-04-06,53.959999,53.970001,53.529999,53.689999,43.755653,4700


I am ordering the ETFs with respective to each country's position in the acronym, i.e, Brazil first, then Russia, and so on until South Africa comes in last. 

This will make it easier to interperate the results when I pass the data into the models that we will be using to create our portfolio. 

In [3]:
countries = ['Brazil', 'Russia', 'India', 'China', 'South Africa']

dfs = [df_brazil, df_russia, df_india, df_china , df_SA, df_hist]

for df in dfs:
    df['Date'] = pd.to_datetime(df['Date'])

The South African data happens to be displayed in GBP currency, whereas all the rest is displayed in USD. Therefore, I'm converting the South African data into USD. 

In [4]:
df_SA = pd.merge(df_SA, pd.DataFrame(df_hist[[' Close', 'Date']]), on='Date')
df_SA.rename(columns={' Close':'Exchange_Rate'}, inplace=True)

for col in df_SA.columns:
    if col != 'Date' and col != 'Volume' and col != 'Country' and col != "Exchange_Rate":
        df_SA[col] = df_SA.apply(lambda row : row[col] * row['Exchange_Rate'], axis=1)

df_SA.drop(columns='Exchange_Rate', inplace=True)

df_SA.to_csv('outSA.csv')
df_SA_out = pd.read_csv('outSA.csv')

df_SA_out.drop(columns='Unnamed: 0', inplace=True)
df_SA_out['Date'] = pd.to_datetime(df_SA_out['Date'])
dfs[4] = df_SA_out

I added the df_hist, a df with the historical exhange rate between GPB and USD for the last 12 years,
into the array dfs. But now that it has served its use, I'm removing it. 

In [5]:
dfs.pop(-1)

,Date,Open,High,Low,Close
0,2022-06-06,1.2489,1.2578,1.2477,1.2533
1,2022-06-03,1.2577,1.2590,1.2484,1.2489
2,2022-06-02,1.2491,1.2587,1.2469,1.2578
3,2022-06-01,1.2602,1.2617,1.2459,1.2487
4,2022-05-31,1.2652,1.2656,1.2560,1.2603
...,...,...,...,...,...
3222,2010-01-28,1.6171,1.6171,1.6138,1.6138
3223,2010-01-27,1.6143,1.6171,1.6143,1.6171
3224,2010-01-26,1.6244,1.6244,1.6143,1.6143
3225,2010-01-25,1.6115,1.6244,1.6115,1.6244


At this point I'm just filling in the blanks for the days that are missing in the datasets. 

What I'm also going to do is add a returns column to each data set

In [6]:
def calc_returns(df, limit):
    
    returns = [1]
    for index, row in df.iterrows():
        if index >= limit-1:
            break

        returns.append(df._get_value(index+1, 'Close')/df._get_value(index, 'Close') - 1)

    df['Returns'] = returns

    return df

In [7]:
for i, df in enumerate(dfs):

    daily_data = pd.DataFrame(pd.date_range(start=df['Date'].min(),end=df['Date'].max()))
    daily_data.rename(columns={ daily_data.columns[0]: "Date" }, inplace = True)
    daily_data['Date'] = pd.to_datetime(daily_data['Date'])

    # Add the missing dates
    df2 = pd.merge(df, daily_data,on='Date',how='outer')
    df2 = df2.sort_values(by=['Date'])

    df3 = df2.interpolate(method='linear', axis=0).ffill().bfill()
    df4 = df3.reset_index()

    # now add a returns column
    dfs[i] = calc_returns(df4, len(df4))

In [8]:
for i in range(len(dfs)):
    df = dfs[i][dfs[i].Date > '2011-03-31']
    dfs[i] = df

for i in range(len(dfs)):
    df = dfs[i][dfs[i].Date < '2022-03-29']
    dfs[i] = df


df_complete = pd.DataFrame()

for i in range(len(dfs)):
    dfs[i].drop(columns='index', inplace=True)
    dfs[i].reset_index(inplace=True)

    for column in dfs[i].columns:
        if column == "Date" or column=='index':
            continue
        
        else:
            dfs[i].rename(columns={column: countries[i]+ ' ' +column}, inplace=True)
    
# now concatenate the dfs into one data frame 
df_concat = pd.concat([df for df in dfs], axis=1)
df_concat.drop(['index', 'Date'], axis=1, inplace=True)
df_concat['Date'] = pd.date_range(start='2011-03-31', periods = len(df_concat))
df_concat['Date'] = pd.to_datetime(df["Date"])

# now rename it back to df
df = df_concat

In [9]:
df_features = (
                df
                .assign(day = df.Date.dt.day)
                .assign(month = df.Date.dt.month)
                .assign(day_of_week = df.Date.dt.dayofweek)
                .assign(week_of_year = df.Date.dt.week)
                .assign(year = df.Date.dt.year)
              )

<ipython-input-9-503732ed2578>:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  .assign(week_of_year = df.Date.dt.week)


In [10]:
def generate_cyclical_features(df, col_name, period, start_num=0):
    kwargs = {
        f'sin_{col_name}' : lambda x: np.sin(2*np.pi*(df[col_name]-start_num)/period),
        f'cos_{col_name}' : lambda x: np.cos(2*np.pi*(df[col_name]-start_num)/period)    
             }
    return df.assign(**kwargs).drop(columns=[col_name])

df_features = generate_cyclical_features(df_features, 'day', 24, 0)
df_features = generate_cyclical_features(df_features, 'day_of_week', 7, 0)
df_features = generate_cyclical_features(df_features, 'month', 12, 1)
df_features = generate_cyclical_features(df_features, 'week_of_year', 52, 0)

df = df_features

In [11]:
df.head()

,Brazil Open,Brazil High,Brazil Low,Brazil Close,Brazil Adj Close,Brazil Volume,Brazil Returns,Russia Open,Russia High,Russia Low,...,Date,year,sin_day,cos_day,sin_day_of_week,cos_day_of_week,sin_month,cos_month,sin_week_of_year,cos_week_of_year
0,78.500000,79.370003,78.070000,79.220001,53.268791,1.606910e+07,0.022062,149.070007,150.550003,149.070007,...,2011-04-01,2011,0.258819,0.965926,-0.433884,-0.900969,1.0,6.123234e-17,1.000000,-1.608123e-16
1,78.846667,79.520002,78.390000,79.406667,53.394311,1.444657e+07,0.002356,150.013336,151.363337,150.013336,...,2011-04-02,2011,0.500000,0.866025,-0.974928,-0.222521,1.0,6.123234e-17,1.000000,-1.608123e-16
2,79.193334,79.670001,78.709999,79.593333,53.519831,1.282403e+07,0.002351,150.956665,152.176671,150.956665,...,2011-04-03,2011,0.707107,0.707107,-0.781831,0.623490,1.0,6.123234e-17,1.000000,-1.608123e-16
3,79.540001,79.820000,79.029999,79.779999,53.645351,1.120150e+07,0.002345,151.899994,152.990005,151.899994,...,2011-04-04,2011,0.866025,0.500000,0.000000,1.000000,1.0,6.123234e-17,0.992709,-1.205367e-01
4,79.449997,80.080002,79.230003,79.510002,53.463795,1.189970e+07,-0.003384,151.160004,152.160004,151.160004,...,2011-04-05,2011,0.965926,0.258819,0.781831,0.623490,1.0,6.123234e-17,0.992709,-1.205367e-01


In [12]:
df.drop(columns='Date', inplace=True)

In [13]:
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

In [14]:
def df_fuller(timeseries, window_size=12, filename=None):    
    #Determing rolling statistics
    plt.figure(figsize=(15,10))
    rolmean = timeseries.rolling(window_size).mean()
    rolstd = timeseries.rolling(window_size).std()
    #Plot rolling statistics:
    plt.plot(timeseries, color='blue',label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    if filename is not None:
        plt.savefig(filename)
    plt.show()

    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC', maxlag=window_size*4)
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [15]:
df.to_csv('clean_data.csv')

In [16]:
df.head()

,Brazil Open,Brazil High,Brazil Low,Brazil Close,Brazil Adj Close,Brazil Volume,Brazil Returns,Russia Open,Russia High,Russia Low,...,South Africa Returns,year,sin_day,cos_day,sin_day_of_week,cos_day_of_week,sin_month,cos_month,sin_week_of_year,cos_week_of_year
0,78.500000,79.370003,78.070000,79.220001,53.268791,1.606910e+07,0.022062,149.070007,150.550003,149.070007,...,0.028368,2011,0.258819,0.965926,-0.433884,-0.900969,1.0,6.123234e-17,1.000000,-1.608123e-16
1,78.846667,79.520002,78.390000,79.406667,53.394311,1.444657e+07,0.002356,150.013336,151.363337,150.013336,...,0.001658,2011,0.500000,0.866025,-0.974928,-0.222521,1.0,6.123234e-17,1.000000,-1.608123e-16
2,79.193334,79.670001,78.709999,79.593333,53.519831,1.282403e+07,0.002351,150.956665,152.176671,150.956665,...,0.001655,2011,0.707107,0.707107,-0.781831,0.623490,1.0,6.123234e-17,1.000000,-1.608123e-16
3,79.540001,79.820000,79.029999,79.779999,53.645351,1.120150e+07,0.002345,151.899994,152.990005,151.899994,...,0.001652,2011,0.866025,0.500000,0.000000,1.000000,1.0,6.123234e-17,0.992709,-1.205367e-01
4,79.449997,80.080002,79.230003,79.510002,53.463795,1.189970e+07,-0.003384,151.160004,152.160004,151.160004,...,0.000786,2011,0.965926,0.258819,0.781831,0.623490,1.0,6.123234e-17,0.992709,-1.205367e-01
